In [ ]:
import pandas as pd
import openpyxl
import numpy as np

In [ ]:
data_14 = pd.read_csv('immo/2014.csv').drop(columns='Unnamed: 0')
data_15 = pd.read_csv('immo/2015.csv').drop(columns='Unnamed: 0')
data_16 = pd.read_csv('immo/2016.csv').drop(columns='Unnamed: 0')
data_17 = pd.read_csv('immo/2017.csv').drop(columns='Unnamed: 0')
data_18 = pd.read_csv('immo/2018.csv').drop(columns='Unnamed: 0')
data_19 = pd.read_csv('immo/2019.csv').drop(columns='Unnamed: 0')
data_20 = pd.read_csv('immo/2020.csv').drop(columns='Unnamed: 0')
data_21 = pd.read_csv('immo/2021.csv').drop(columns='Unnamed: 0')


In [ ]:
data = pd.concat([data_14, data_15, data_16, data_17, data_18, data_19, data_20, data_21])
print(data.shape)
data = data.rename(columns={"INSEE_COM" : "Code INSEE"})
data.head()

In [ ]:
commune_df = pd.read_csv("immo/correspondance-code-insee-code-postal.csv", sep=';', usecols=["Code INSEE", "Commune"])
commune_df.head()

In [ ]:
data = data.merge(commune_df, how='left', on='Code INSEE').dropna(subset=["Code INSEE"])
data

In [ ]:
data.loc[data["Code INSEE"] == "75056", "Commune"] = "PARIS"
data.loc[data["Code INSEE"] == "13055", "Commune"] = "MARSEILLE"

In [ ]:
# Detect the encoding of the file
import chardet

file_path = 'immo/gares-tgv.csv'
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
encoding = result['encoding']
encoding

In [ ]:
# TGV
tgv = pd.read_csv('immo/gares-tgv.csv', encoding=encoding).drop(columns='Unnamed: 0')
tgv_list = tgv['Nom_Gare'].unique()
print(len(tgv_list))
tgv.head()

In [ ]:
gares = pd.read_csv("immo/liste-des-gares.csv", sep = ";", usecols=["libelle", "code_ligne", "commune", "voyageurs"]).rename(columns={"libelle" : "Nom_gare",
                                                                                                                    "commune" : "Commune"})
gares = gares[gares["voyageurs"] == "O"].reset_index(drop=True)
print(gares.shape)
gares.head()

In [ ]:
gares.isna().sum()

In [ ]:
gares = gares.merge(commune_df, how='left', on='Commune').dropna(subset=["Code INSEE"]).reset_index(drop=True)
gares.head()

In [ ]:
gares.isna().sum()

In [ ]:
for index, el in enumerate(gares["Nom_gare"]):
    if el in tgv_list:
        gares.loc[index, "TGV"] = "Gare TGV"
    else:
        gares.loc[index, "TGV"] = "Gare voyageurs non TGV"

gares.drop(columns=["Commune", "voyageurs", "code_ligne"], inplace=True)
gares.tail()

In [ ]:
gares[gares["TGV"] == "Gare voyageurs non TGV"].head()

In [ ]:
print(gares.shape)
gares["TGV"].unique()

In [ ]:
# merge avec data immo

data = data.merge(gares, how='left', on="Code INSEE")
data['TGV'] = data['TGV'].fillna("Pas de gare")
data

In [ ]:
data["TGV"].value_counts()

In [ ]:
data.loc[data["Code INSEE"] == "75056", :]

In [ ]:
ville = [
    "75056",  # Paris
    "69123",  # Lyon
    "13055",  # Marseille
    "33063",  # Bordeaux
    "59350",  # Lille
    "31555",  # Toulouse
    "44109",  # Nantes
    "67482",  # Strasbourg
    "35238",  # Rennes
    "06088",  # Nice
    "34172",  # Montpellier
    "38185",  # Grenoble
    "76351",  # Le Havre
    "51454",  # Reims
    "76540"   # Rouen
]



data[data["Code INSEE"].isin(ville)]
for index, el in enumerate(data["Code INSEE"]):
    if el in ville:
        data.loc[index, "TGV"] = "Gare TGV"


In [ ]:
# On prends les départements
data["code"] = data["Code INSEE"].str[:2]
data.dropna(subset=['code'], inplace=True)

In [ ]:
data.to_csv("TGV.csv", sep = ";", index=False)

## Génération de la carte intéractive

In [ ]:
import folium

paris = folium.Map(location = [46.8534, 2.3488], zoom_start=6,tiles = "https://{s}.tile.openstreetmap.fr/osmfr/{z}/{x}/{y}.png",
                  attr = '&copy; OpenStreetMap France | &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')

In [ ]:
print(df.shape)

In [ ]:
df = data[data["Annee"] == 2021]
count = df.groupby(["code", "TGV"]).size().to_frame().reset_index().rename(columns={0 : "Occurences"})
df = df.groupby(by=["code"]).agg(Nb_mutations =("Nb_mutations", 'sum'), Prixm2Moyen = ('Prixm2Moyen', 'mean'),
                                 SurfaceMoy = ('SurfaceMoy', 'mean'))

count = count.pivot_table(index='code', columns='TGV', values='Occurences').reset_index()
df = df.merge(count, how='left', on='code')
df

In [ ]:
# Correction nombre de gares à paris
for index, el in enumerate(df["code"]):
    if el == "75":
        df.loc[index, "Gare TGV"] = 4

In [ ]:
# Augmenter le geojson

import json

with open('immo/departements.geojson', 'r') as file:
    geojson_data = json.load(file)

for feature in geojson_data['features']:
    code = feature['properties']['code']
    # Ajouter la propriété "TGV" en fonction de la valeur correspondante dans votre DataFrame
    if code in df['code'].values:
        tgv_value = df.loc[df['code'] == code, 'Gare TGV'].iloc[0]
        feature['properties']['Gare TGV'] = tgv_value

        gare_voyageurs = df.loc[df['code'] == code, 'Gare voyageurs non TGV'].iloc[0]
        feature['properties']['Gare voyageurs non TGV'] = gare_voyageurs
    else:
        feature['properties']['Gare TGV'] = "Aucune données disponibles"
        feature['properties']['Gare voyageurs non TGV'] = "Aucune données disponibles"

with open('immo/modified_departements.geojson', 'w') as file:
    json.dump(geojson_data, file)

In [ ]:

paris = folium.Map(location = [46.8534, 2.3488], zoom_start=6, min_zoom=4, tiles = "https://{s}.tile.openstreetmap.fr/osmfr/{z}/{x}/{y}.png",
        attr = '&copy; OpenStreetMap France | &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')

choropleth = folium.Choropleth(
geo_data= "immo/modified_departements.geojson",
name="choropleth",
data= df,
columns=["code", "Prixm2Moyen"],
key_on="feature.properties.code",
fill_color="YlOrRd",
fill_opacity=0.7,
line_opacity=0.5,
highlight = True,
)
choropleth.geojson.add_to(paris)

result_indexed = df.set_index('code')

for feature in choropleth.geojson.data['features']:
    Communes = feature['properties']['code']
    feature['properties']['Prixm2Moyen'] = '{:,.0f} €'.format(result_indexed.loc[Communes, 'Prixm2Moyen']).replace(',', ' ') if Communes in list(result_indexed.index) else 'Aucune données disponibles'
    feature['properties']['Nb_mutations'] = format(result_indexed.loc[Communes, 'Nb_mutations']) if Communes in list(result_indexed.index) else 'Aucune données disponibles'
    feature['properties']['SurfaceMoy'] = '{:,.0f} m2'.format(result_indexed.loc[Communes, 'SurfaceMoy']) if Communes in list(result_indexed.index) else 'Aucune données disponibles'
    

folium.GeoJsonTooltip(['nom', "Prixm2Moyen", 'SurfaceMoy', 'Nb_mutations', "Gare TGV", "Gare voyageurs non TGV" ]).add_to(choropleth.geojson)    

In [ ]:
paris.save("map.html")

In [ ]:
paris